In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("new") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
df_path = "/home/rupeshr/Documents/split_Test.csv"
df =spark.read.format("com.databricks.spark.csv")\
                    .option("multiline", "true")\
                    .option("header", "true")\
                    .option("inferSchema", "true")\
                    .option("sep", ',')\
                    .option('escape', '\"')\
                    .option("allowSingleQuotes", "true")\
                    .option("ignoreLeadingWhiteSpace", "true")\
                    .option("ignoreTrailingWhiteSpace", "true")\
                    .load(df_path)
df.printSchema()
df.show()

root
 |-- column: string (nullable = true)

+------+
|column|
+------+
|    12|
|    14|
|   abc|
|   efg|
|     2|
|   qwe|
| 12ews|
|   aws|
|    34|
|    56|
|  said|
+------+



In [ ]:
df['Street Number'] = [street_num[0] if any(i.isdigit() for i in column) else 'N/A' for street_num in df.Address.apply(lambda s: s.split(" ",1))]

In [8]:
def is_digit(value):
    if value:
        return value.isdigit()
    else:
        return False

is_digit_udf = udf(is_digit, BooleanType())

df = df.withColumn('numerical', when(is_digit_udf(df['column']), df['column']))
df = df.withColumn('categorical', when(~is_digit_udf(df['column']), df['column']))
df.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 477, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.7 than that in driver 3.6, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


In [8]:
df.withColumn("parsed_url", explode(split(expr("parse_url(URL, 'QUERY')"), "&"))) \
    .withColumn("parsed_url", split("parsed_url", "=")) \
    .select("URL",
            col("parsed_url").getItem(0).alias("param_name"),
            col("parsed_url").getItem(1).alias("value")
            ) \
    .groupBy("URL") \
    .pivot("param_name") \
    .agg(first("value")) \
    .show()

+--------------------+--------+-------------+-------+---------------+-----------+--------------------+------+----------+----------+---------+----------+
|                 URL|Username|analysis_name|coltype|columnparameter|extracttype|                file|module|operations|      type|   values|   version|
+--------------------+--------+-------------+-------+---------------+-----------+--------------------+------+----------+----------+---------+----------+
|http://0.0.0.0:80...|  rupesh|   csv_upload|   null|       c2,c4,c5|       null|ElectricProductio...|    ts|  sum,mean|statistics|    c1,c3|Eversion53|
|http://0.0.0.0:80...|  rupesh|   csv_upload|    num|             c2|       user|MallCustomers_14.csv|    da|   box_cox|      null|ageboxcox|Mversion23|
|http://0.0.0.0:80...|  rupesh|   csv_upload|   null|           null|       null|ElectricProductio...|    da|      null|      null|     null|Eversion53|
+--------------------+--------+-------------+-------+---------------+-----------+-

In [12]:
var="email"
df=df.withColumn("parsed_url", explode(split(expr("parse_url("+var+", 'SCHEME')"), "@")))
df.show()

+-----+----------+
|email|parsed_url|
+-----+----------+
+-----+----------+



In [122]:
df=df.withColumn("parsed_url", split("parsed_url", "="))
df.show()

+--------------------+--------------------+
|                 URL|          parsed_url|
+--------------------+--------------------+
|http://0.0.0.0:80...|[analysis_name, c...|
|http://0.0.0.0:80...|[file, ElectricPr...|
|http://0.0.0.0:80...|[version, Eversio...|
|http://0.0.0.0:80...|  [Username, rupesh]|
|http://0.0.0.0:80...|        [module, da]|
|http://0.0.0.0:80...|[file, ElectricPr...|
|http://0.0.0.0:80...|  [type, statistics]|
|http://0.0.0.0:80...|  [Username, rupesh]|
|http://0.0.0.0:80...|[analysis_name, c...|
|http://0.0.0.0:80...|[version, Eversio...|
|http://0.0.0.0:80...|        [module, ts]|
|http://0.0.0.0:80...|[columnparameter,...|
|http://0.0.0.0:80...|[operations, sum,...|
|http://0.0.0.0:80...|     [values, c1,c3]|
|http://0.0.0.0:80...|        [module, da]|
|http://0.0.0.0:80...|[file, MallCustom...|
|http://0.0.0.0:80...|  [Username, rupesh]|
|http://0.0.0.0:80...|[version, Mversio...|
|http://0.0.0.0:80...|[analysis_name, c...|
|http://0.0.0.0:80...|      [col

In [123]:
df=df.select("URL",col("parsed_url").getItem(0).alias("param_name"),col("parsed_url").getItem(1).alias("value"))
df.show()

+--------------------+---------------+--------------------+
|                 URL|     param_name|               value|
+--------------------+---------------+--------------------+
|http://0.0.0.0:80...|  analysis_name|          csv_upload|
|http://0.0.0.0:80...|           file|ElectricProductio...|
|http://0.0.0.0:80...|        version|          Eversion53|
|http://0.0.0.0:80...|       Username|              rupesh|
|http://0.0.0.0:80...|         module|                  da|
|http://0.0.0.0:80...|           file|ElectricProductio...|
|http://0.0.0.0:80...|           type|          statistics|
|http://0.0.0.0:80...|       Username|              rupesh|
|http://0.0.0.0:80...|  analysis_name|          csv_upload|
|http://0.0.0.0:80...|        version|          Eversion53|
|http://0.0.0.0:80...|         module|                  ts|
|http://0.0.0.0:80...|columnparameter|            c2,c4,c5|
|http://0.0.0.0:80...|     operations|            sum,mean|
|http://0.0.0.0:80...|         values|  

In [117]:
for i in df.select("param_name").collect():
    print(i)
    df= df.na.replace(to_replace=i[0],value='URL'+'_'+i[0],subset=['param_name'])
df.show()

Row(param_name='analysis_name')
Row(param_name='file')
Row(param_name='version')
Row(param_name='Username')
Row(param_name='module')
Row(param_name='file')
Row(param_name='type')
Row(param_name='Username')
Row(param_name='analysis_name')
Row(param_name='version')
Row(param_name='module')
Row(param_name='columnparameter')
Row(param_name='operations')
Row(param_name='values')
Row(param_name='module')
Row(param_name='file')
Row(param_name='Username')
Row(param_name='version')
Row(param_name='analysis_name')
Row(param_name='coltype')
Row(param_name='columnparameter')
Row(param_name='extracttype')
Row(param_name='operations')
Row(param_name='values')


KeyboardInterrupt: 

In [125]:
df=df.groupBy("URL").pivot("param_name").agg(first("value"))

In [127]:
df.columns 

['URL',
 'Username',
 'analysis_name',
 'coltype',
 'columnparameter',
 'extracttype',
 'file',
 'module',
 'operations',
 'type',
 'values',
 'version']

In [130]:
s=["URL"+'_'+col_name for col_name in df.columns]

In [138]:
df=df.select([col(c).alias("URL"+"_" + c) for c in df.columns])

In [139]:
df.columns

['URL_URL',
 'URL_Username',
 'URL_analysis_name',
 'URL_coltype',
 'URL_columnparameter',
 'URL_extracttype',
 'URL_file',
 'URL_module',
 'URL_operations',
 'URL_type',
 'URL_values',
 'URL_version']

In [142]:
df = df.withColumnRenamed('URL_URL', 'URL')
df.show()

+--------------------+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|                 URL|URL_Username|URL_analysis_name|URL_coltype|URL_columnparameter|URL_extracttype|            URL_file|URL_module|URL_operations|  URL_type|URL_values|URL_version|
+--------------------+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|http://0.0.0.0:80...|      rupesh|       csv_upload|       null|           c2,c4,c5|           null|ElectricProductio...|        ts|      sum,mean|statistics|     c1,c3| Eversion53|
|http://0.0.0.0:80...|      rupesh|       csv_upload|        num|                 c2|           user|MallCustomers_14.csv|        da|       box_cox|      null| ageboxcox| Mversion23|
|http://0.0.0.0:80...|      rupesh|       csv_upload|       null|               null|

In [11]:
def HTTP_to_column(df,url_column):
    d=df
    df=df.withColumn("parsed_url", explode(split(expr("parse_url("+url_column+", 'QUERY')"), "&"))) \
    .withColumn("parsed_url", split("parsed_url", "=")) \
    .select(url_column ,col("parsed_url").getItem(0).alias("param_name"),col("parsed_url").getItem(1).alias("value")) \
    .groupBy(url_column).pivot("param_name").agg(first("value"))
    df=df.select([col(c).alias(url_column+"_" + c) for c in df.columns])
    df = df.withColumnRenamed(url_column+"_"+url_column, url_column)
    d=d.join(df, url_column)
    return df

In [12]:
df_path ="/home/rupeshr/Desktop/TSA_Python/sample_url.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+--------------------+-----+
|                 URL|count|
+--------------------+-----+
|http://0.0.0.0:80...|   56|
|http://0.0.0.0:80...|   76|
|http://0.0.0.0:80...|   89|
+--------------------+-----+



In [9]:
df.dtypes

[('URL', 'string'), ('count', 'int')]

In [13]:
s=HTTP_to_column(df,"URL")
s.show()

+--------------------+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|                 URL|URL_Username|URL_analysis_name|URL_coltype|URL_columnparameter|URL_extracttype|            URL_file|URL_module|URL_operations|  URL_type|URL_values|URL_version|
+--------------------+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|http://0.0.0.0:80...|      rupesh|       csv_upload|       null|           c2,c4,c5|           null|ElectricProductio...|        ts|      sum,mean|statistics|     c1,c3| Eversion53|
|http://0.0.0.0:80...|      rupesh|       csv_upload|        num|                 c2|           user|MallCustomers_14.csv|        da|       box_cox|      null| ageboxcox| Mversion23|
|http://0.0.0.0:80...|      rupesh|       csv_upload|       null|               null|

In [11]:
df=df.join(s, 'URL')
df.show()

+--------------------+-----+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|                 URL|count|URL_Username|URL_analysis_name|URL_coltype|URL_columnparameter|URL_extracttype|            URL_file|URL_module|URL_operations|  URL_type|URL_values|URL_version|
+--------------------+-----+------------+-----------------+-----------+-------------------+---------------+--------------------+----------+--------------+----------+----------+-----------+
|http://0.0.0.0:80...|   76|      rupesh|       csv_upload|       null|           c2,c4,c5|           null|ElectricProductio...|        ts|      sum,mean|statistics|     c1,c3| Eversion53|
|http://0.0.0.0:80...|   89|      rupesh|       csv_upload|        num|                 c2|           user|MallCustomers_14.csv|        da|       box_cox|      null| ageboxcox| Mversion23|
|http://0.0.0.0:80...|   56|      rupesh|       csv_upl

In [7]:

import pandas as pd
pd.options.display.max_columns = None

In [8]:
s.show()

+--------------------+----------+---------+----------+--------+----------+----------+----------+----------+-------------+----------+-----------+--------+----------+--------+--------+
|                 URL|URL_amount|URL_cost2|URL_member|URL_name|URL_param1|URL_param2|URL_param3|URL_param4|URL_parameter|URL_params|URL_product|URL_rent|URL_source|URL_test|URL_user|
+--------------------+----------+---------+----------+--------+----------+----------+----------+----------+-------------+----------+-----------+--------+----------+--------+--------+
|https://www.examp...|      null|     null|      null|     api|      null|      null|      null|      null|         null|      null|       null|    null|      null|    null|    test|
|https://www.examp...|      null|     null|      null|    user|      null|      null|      null|      null|         null|      null|          3|    null|      null|    user|    null|
|https://www.examp...|      null|     null|      null|    null|         2|         b|

In [4]:
from pyspark.sql import functions as F

split_email = F.split(df.email, "@")

In [7]:
print(split_email)

Column<b'split(email, @, -1)'>


In [9]:
df=df.withColumn("email", split("email", "@"))
df.show()

+--------------------+
|               email|
+--------------------+
|[rupeshrupeesgmai...|
| [kokojambo, mon.eu]|
|      [lol, fsa.org]|
|      [ii, koko.com]|
+--------------------+



In [7]:
def split_email(df, email_column):
    df=df.select(email_column,split(email_column,"@").getItem(0).alias("username"),split(email_column,"@").getItem(1).alias("domain"))
    return df


In [6]:
df_path ="/home/rupeshr/Desktop/TSA_Python/sample_email.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+--------------------+
|               email|
+--------------------+
|rupeshrupees@gmai...|
|    kokojambo@mon.eu|
|         lol@fsa.org|
|        ii@koko.com |
|rupesh.subramani@...|
+--------------------+



In [8]:
s=split_email(df, "email")
s.show()

+--------------------+----------------+---------+
|               email|        username|   domain|
+--------------------+----------------+---------+
|rupeshrupees@gmai...|    rupeshrupees|gmail.com|
|    kokojambo@mon.eu|       kokojambo|   mon.eu|
|         lol@fsa.org|             lol|  fsa.org|
|        ii@koko.com |              ii|koko.com |
|rupesh.subramani@...|rupesh.subramani|gmail.com|
+--------------------+----------------+---------+



In [10]:
df=df.join(s, 'email')
df.show()

+--------------------+----------------+---------+
|               email|        username|   domain|
+--------------------+----------------+---------+
|rupeshrupees@gmai...|    rupeshrupees|gmail.com|
|    kokojambo@mon.eu|       kokojambo|   mon.eu|
|         lol@fsa.org|             lol|  fsa.org|
|        ii@koko.com |              ii|koko.com |
|rupesh.subramani@...|rupesh.subramani|gmail.com|
+--------------------+----------------+---------+



In [1]:
import matplotlib.pyplot as plt

In [78]:
from pyspark.sql.types import *
from pyspark.ml.feature import *
from pyspark.sql.functions import UserDefinedFunction, udf
from pyspark.sql.functions import lit
import ast
import numpy as np
import traceback
from math import ceil,floor
import plotly.express as px

In [105]:
df_path ="/home/rupeshr/Downloads/Mall_Customers.csv"
df = spark.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load(df_path)
df.show()

+----------+------+---+------------------+----------------------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
|         6|Female| 22|                17|                    76|
|         7|Female| 35|                18|                     6|
|         8|Female| 23|                18|                    94|
|         9|  Male| 64|                19|                     3|
|        10|Female| 30|                19|                    72|
|        11|  Male| 67|                19|                    14|
|        12|Female| 35|                19|                    99|
|        1

In [95]:
val = df.groupby().max('Age').collect()[0]
vals = df.groupby().min('Age').collect()[0]
c = [val[0]] # max
d = [vals[0]] # min
print(c, d)

[70] [18]


In [70]:
splits=d + [25,30,40,50] + c
splits_dict = {i:splits[i] for i in range(len(splits))}
print(splits_dict)

{0: 18, 1: 25, 2: 30, 3: 40, 4: 50, 5: 70}


In [106]:
x=[10,20,30,40,50,60,70]
bucketizer = Bucketizer(splits=x, inputCol="Age",outputCol="age_result")

In [107]:
bucketed = bucketizer.transform(df)
bucketed.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|age_result|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|       0.0|
|         2|  Male| 21|                15|                    81|       1.0|
|         3|Female| 20|                16|                     6|       1.0|
|         4|Female| 23|                16|                    77|       1.0|
|         5|Female| 31|                17|                    40|       2.0|
|         6|Female| 22|                17|                    76|       1.0|
|         7|Female| 35|                18|                     6|       2.0|
|         8|Female| 23|                18|                    94|       1.0|
|         9|  Male| 64|                19|                     3|       5.0|
|        10|Female| 30|                19|                    72|       2.0|

In [98]:
bucketed = bucketed.replace(to_replace=splits_dict, subset=['age_result'])
bucketed.show()

+----------+------+---+------------------+----------------------+----------+
|CustomerID| Genre|Age|Annual Income (k$)|Spending Score (1-100)|age_result|
+----------+------+---+------------------+----------------------+----------+
|         1|  Male| 19|                15|                    39|      18.0|
|         2|  Male| 21|                15|                    81|      25.0|
|         3|Female| 20|                16|                     6|      25.0|
|         4|Female| 23|                16|                    77|      25.0|
|         5|Female| 31|                17|                    40|      30.0|
|         6|Female| 22|                17|                    76|      25.0|
|         7|Female| 35|                18|                     6|      30.0|
|         8|Female| 23|                18|                    94|      25.0|
|         9|  Male| 64|                19|                     3|      70.0|
|        10|Female| 30|                19|                    72|      30.0|

In [108]:
s = []
for i in range(len(x[:-1])):
    q = (np.round(x[i],6), np.round(x[i+1],6))
    s.append(q)

In [109]:
print(s)

[(10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70)]


In [118]:
def testUDF(col, s):
    if col:
        s = ast.literal_eval(str(s))
        # s = [(17, 35), (35, 44), (44, 92)]
        return str(s[int(col)])
    else:
        return
            # myudf = UserDefinedFunction(testUDF, StringType())
myudf = udf(testUDF, StringType())
f = bucketed.withColumn('age_source', myudf('age_result', lit(str(s))))
sd = f.drop("age_result")

In [119]:
sd

DataFrame[CustomerID: int, Genre: string, Age: int, Annual Income (k$): int, Spending Score (1-100): int, age_source: string]